In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import pingouin as pg
import scipy.stats
from scipy.stats import multivariate_normal as mvn

In [2]:
##getting rid of convergence warnings
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

<h3>Within-subject correlation parameter</h3>

<p>The within-subject correlation parameter for the initial power analysis was set at 0.5. This was based off of standard convention (including the g*power default setting) due to concerns about the reliability of n=11 pilot data </p>
<p>However, as an additional check, we opted to assess the within-subject correlation in the pilot data, and on the basis of this check, run some power calculations using a more conservative within-subject correlation estimates </P>

In [3]:
## load in the pilot data
task_summary=pd.read_csv("csvs/pilot_data_for_power.csv")
task_summary=task_summary.replace(['Disgust', 'Fear', 'Points'], [1.0,2.0,3.0])

In [4]:
#transform outcome variables as they are skewed
    #analysis plan specifies yeojoohnson transforms for skewed outcome variables
task_summary['regressive_er_transformed'] = scipy.stats.yeojohnson(task_summary.mean_regressive_er)[0]
task_summary['perseverative_er_transformed']= scipy.stats.yeojohnson(task_summary.mean_perseverative_er)[0]

In [5]:
#check within subject correlation in pilot data - regressive error
pg.rm_corr(data=task_summary, x='regressive_er_transformed', y='block_type', subject='participant_no')
    #finds a significant within subject correlation of 0.53 (p=0.008)

,r,dof,pval,CI95%,power
rm_corr,0.534214,21,0.008644,"[0.16, 0.78]",0.774637


In [6]:
#check within subject correlation in pilot data
pg.rm_corr(data=task_summary, x='perseverative_er_transformed', y='block_type', subject='participant_no')
    #no significant within subject correlation

,r,dof,pval,CI95%,power
rm_corr,-0.129557,21,0.555743,"[-0.51, 0.3]",0.090593


In [7]:
#check power from initial power analysis with a more conservative estimate
    #within-subject corr = 0.25 (half initial estimate)

In [8]:
#use the same function as in the initial power analysis
    #only alteration is ability to change effect size and within subject correlation parameters
def original_mixed_effects_power_analysis(num_subjects, num_sims,  effect_size, within_subject_corr):
    sim_table=pd.DataFrame()
    for n in range(num_sims):
        #simulate data
        std_dev=1
        conditions=['Disgust', 'Fear', 'Points']
        mean_diff=effect_size*std_dev
        means=[1+mean_diff, 1, 1]
        cov_matrix=np.eye(len(conditions))*(1-within_subject_corr)+within_subject_corr
        scores=mvn.rvs(mean=means, cov=cov_matrix, size=num_subjects)
        df=pd.DataFrame(data=scores, columns=['Disgust', 'Fear', 'Points'])
        df['participant_no']=df.index+1
        df=pd.melt(df, id_vars=['participant_no'], var_name='Condition', value_name='Perseveration') ##convert to long form

        #run mixed effects model on simulated data
        formula = 'Perseveration ~ Condition'
        md=smf.mixedlm(formula, df, groups=df['participant_no'], missing='drop')
        results=md.fit()
        
        #is it significant
        if results.pvalues['Condition[T.Fear]'] < alpha:
            result="Sig"
        else:
            result="Non Sig"
        
        #save into a table
        sim_row=pd.DataFrame({'simulation': [n], 'result': [result]})
        sim_table=pd.concat([sim_table, sim_row])
    return sim_table

In [9]:
# within subject corr =0.25
alpha=0.05
effect_size=0.2
power=0.95
num_subjects=323
num_sims=10000
within_subject_corr=0.25
sim_table=pd.DataFrame()

sim_table=original_mixed_effects_power_analysis(num_subjects, num_sims, effect_size, within_subject_corr)
power=len(sim_table[sim_table.result=='Sig'].simulation)/len(sim_table.simulation)
print(power)

0.8289


In [10]:
# within subject corr = 0 (very conservative)
alpha=0.05
effect_size=0.2
power=0.95
num_subjects=323
num_sims=10000
within_subject_corr=0

sim_table=original_mixed_effects_power_analysis(num_subjects, num_sims, effect_size, within_subject_corr)
power=len(sim_table[sim_table.result=='Sig'].simulation)/len(sim_table.simulation)
print(power)

0.7213


In [11]:
# within subject corr = 0 (very conservative)
    #slightly larger effect size
alpha=0.05
effect_size=0.25
power=0.95
num_subjects=323
num_sims=10000
within_subject_corr=0

sim_table=original_mixed_effects_power_analysis(num_subjects, num_sims, effect_size, within_subject_corr)
power=len(sim_table[sim_table.result=='Sig'].simulation)/len(sim_table.simulation)
print(power)

0.8884
